In [1]:
import pandas as pd
import feather

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.models import load_model


Using TensorFlow backend.


In [2]:
n_groups_per_col = 30
data_train = pd.read_feather("../data/models/ind/binary/data_train.feather")

labels_low_train = pd.read_feather("../data/models/ind/binary/labels_low_train.feather")
labels_high_train = pd.read_feather("../data/models/ind/binary/labels_high_train.feather")
labels_close_train = pd.read_feather("../data/models/ind/binary/labels_close_train.feather")

In [3]:
train_data = data_train.to_numpy()
labels_low_train = labels_low_train.to_numpy()
labels_high_train = labels_high_train.to_numpy()
labels_close_train = labels_close_train.to_numpy()

In [7]:
def fit_ind_binary_model(data_train, labels_train):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(data_train.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(labels_train.shape[1], activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(data_train, labels_train, validation_split=0.2, batch_size=512, epochs=20, verbose=1)
    
    return (history, model)

In [8]:
# low
(low_history, low_model) = fit_ind_binary_model(data_train, labels_low_train)
feather.write_dataframe(pd.DataFrame.from_dict(low_history.history), "../data/models/ind/binary/low_history.feather", compression = "uncompressed")
low_model.save("../data/models/ind/binary/low_model.hdf5")


Train on 4342137 samples, validate on 1085535 samples
Epoch 1/20
4342137/4342137 [==============================] - 36s 8us/step - loss: 0.4319 - accuracy: 0.7927 - val_loss: 0.3906 - val_accuracy: 0.8138
Epoch 2/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4043 - accuracy: 0.8064 - val_loss: 0.3880 - val_accuracy: 0.8143
Epoch 3/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.3962 - accuracy: 0.8107 - val_loss: 0.3812 - val_accuracy: 0.8208
Epoch 4/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4089 - accuracy: 0.8049 - val_loss: 0.4114 - val_accuracy: 0.8031
Epoch 5/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.3968 - accuracy: 0.8118 - val_loss: 0.4100 - val_accuracy: 0.8043
Epoch 6/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.3922 - accuracy: 0.8135 - val_loss: 0.3997 - val_accuracy: 0.8098
Epoch 7/20
4342137/4342137 [============

In [9]:
# high
(high_history, high_model) = fit_ind_binary_model(train_data, labels_high_train)
feather.write_dataframe(pd.DataFrame.from_dict(high_history.history), "../data/models/ind/binary/high_history.feather", compression = "uncompressed")
high_model.save("../data/models/ind/binary/high_model.hdf5")

Train on 4342137 samples, validate on 1085535 samples
Epoch 1/20
4342137/4342137 [==============================] - 36s 8us/step - loss: 0.4299 - accuracy: 0.7932 - val_loss: 0.3995 - val_accuracy: 0.8091
Epoch 2/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4031 - accuracy: 0.8065 - val_loss: 0.3791 - val_accuracy: 0.8192
Epoch 3/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.3976 - accuracy: 0.8088 - val_loss: 0.4191 - val_accuracy: 0.7920
Epoch 4/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.3963 - accuracy: 0.8091 - val_loss: 0.4042 - val_accuracy: 0.8072
Epoch 5/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.3946 - accuracy: 0.8105 - val_loss: 0.3765 - val_accuracy: 0.8197
Epoch 6/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.3908 - accuracy: 0.8125 - val_loss: 0.3762 - val_accuracy: 0.8198
Epoch 7/20
4342137/4342137 [============

In [10]:
# close
(close_history, close_model) = fit_ind_binary_model(train_data, labels_close_train)
feather.write_dataframe(pd.DataFrame.from_dict(close_history.history), "../data/models/ind/binary/history.feather", compression = "uncompressed")
close_model.save("../data/models/ind/binary/close_model.hdf5")


Train on 4342137 samples, validate on 1085535 samples
Epoch 1/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4673 - accuracy: 0.7697 - val_loss: 0.4604 - val_accuracy: 0.7718
Epoch 2/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4507 - accuracy: 0.7788 - val_loss: 0.4444 - val_accuracy: 0.7827
Epoch 3/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4482 - accuracy: 0.7805 - val_loss: 0.4417 - val_accuracy: 0.7847
Epoch 4/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4465 - accuracy: 0.7816 - val_loss: 0.4604 - val_accuracy: 0.7711
Epoch 5/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4457 - accuracy: 0.7821 - val_loss: 0.4383 - val_accuracy: 0.7872
Epoch 6/20
4342137/4342137 [==============================] - 35s 8us/step - loss: 0.4449 - accuracy: 0.7826 - val_loss: 0.4401 - val_accuracy: 0.7853
Epoch 7/20
4342137/4342137 [============

In [ ]:
### Predict

# low_model = load_model("../data/models/ind/binary/low_model.hdf5")
# high_model = load_model("../data/models/ind/binary/high_model.hdf5")
# close_model = load_model("../data/models/ind/binary/close_model.hdf5")

In [11]:
data_test = pd.read_feather("../data/models/ind/binary/data_test.feather").to_numpy()

low_pred_prob = low_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(low_pred_prob), "../data/models/ind/binary/low_pred_prob.feather", compression='uncompressed')

high_pred_prob = high_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(high_pred_prob), "../data/models/ind/binary/high_pred_prob.feather", compression='uncompressed')

close_pred_prob = close_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(close_pred_prob), "../data/models/ind/binary/close_pred_prob.feather", compression='uncompressed')
